In [ ]:

import geopandas as gpd
import holoviews as hv
import numpy as np
import pandas as pd
import pyproj
import stackstac
import xarray as xr
from shapely.geometry import mapping
import rioxarray as rio
import hvplot.xarray
import hvplot.pandas


In [ ]:
# from dask.distributed import Client, progress, LocalCluster
# cluster = LocalCluster()
# client = Client(cluster)

In [ ]:
aoi = gpd.read_file('data/catchment_outline.geojson', crs="EPGS:4326")

In [ ]:
aoi_geojson = mapping(aoi.iloc[0].geometry)

In [ ]:
import pystac_client

URL = "https://earth-search.aws.element84.com/v1"
catalog = pystac_client.Client.open(URL)

In [ ]:
items = catalog.search(
    intersects=aoi_geojson,
    collections=["sentinel-2-l2a"],
    datetime="2019-02-01/2019-06-10"
).item_collection()
len(items)

In [ ]:
selected_item = items[1]

In [ ]:
for key, asset in selected_item.assets.items():
    print(f"{key}: {asset.title}")

In [ ]:
ds = stackstac.stack(items)

In [ ]:
green = ds.sel(band='green')
swir = ds.sel(band='swir16')
scl = ds.sel(band='scl')

In [ ]:
ndsi = (green - swir) / (green + swir)

In [ ]:
snow = xr.where((ndsi > 0.42) & ~np.isnan(ndsi), 1, ndsi)

In [ ]:
snowmap = xr.where((snow <= 0.42) & ~np.isnan(snow), 0, snow)

In [ ]:
# mask = (scl != 8) & (scl != 9) & (scl != 3) 
mask = np.logical_not(scl.isin([8, 9, 3]))  # more elegant but not sure about it from a teaching perspective

In [ ]:
snow_cloud = xr.where(mask, snowmap, 2)

In [ ]:
aoi_utm32 = aoi.to_crs(epsg=32632)

In [ ]:
geom_utm32 = aoi_utm32.iloc[0]['geometry']

In [ ]:
snow_cloud.rio.write_crs("EPSG:32632", inplace=True)
snow_cloud.rio.set_nodata(np.nan, inplace=True)

In [ ]:
snowmap_clipped = snow_cloud.rio.clip([geom_utm32])

In [ ]:
from dask.diagnostics import ProgressBar

with ProgressBar():
    clipped = snowmap_clipped.compute()

In [ ]:
clipped_date = clipped.groupby(clipped.time.dt.floor('D')).max(skipna=True)

In [ ]:
clipped_date = clipped_date.rename({'floor': 'date'})

In [ ]:
clipped_date.hvplot.image(
    x='x',
    y='y',
    groupby='date',
    crs=pyproj.CRS.from_epsg(32632),
    cmap='Pastel2',
    clim=(-1, 2),
    frame_width=500,
    frame_height=500,
    title='Snowmap',
    geo=True, tiles='OSM')

In [ ]:
cloud = xr.where(clipped_date == 2, 1, np.nan).count(dim=['x', 'y'])

In [ ]:
aot_total = clipped_date.count(dim=['x', 'y'])

In [ ]:
cloud_fraction = cloud / aot_total * 100

In [ ]:
cloud_fraction.hvplot.line(title='Cloud cover %') * hv.HLine(25).opts(
    color='red',
    line_dash='dashed',
    line_width=2.0,
)

In [ ]:
snow = xr.where(clipped_date == 1, 1, np.nan).count(dim=['x', 'y'])
snow_fraction = snow / aot_total * 100

In [ ]:
snow_fraction.hvplot.line(title='Snow cover area (%)')

In [ ]:
masked_cloud_fraction = cloud_fraction < 30

In [ ]:
snow_selected = snow_fraction.sel(date=masked_cloud_fraction)

In [ ]:
snow_selected.hvplot.line(title="Snow fraction")

In [ ]:
discharge_ds = pd.read_csv('data/ADO_DSC_ITH1_0025.csv', sep=',', index_col='Time', parse_dates=True)
discharge_ds.head()

In [ ]:
start_date = pd.to_datetime("2019/02/01")
end_date = pd.to_datetime("2019/06/30")
# filter discharge data to start and end dates
discharge_ds = discharge_ds.loc[start_date:end_date]

discharge_ds.discharge_m3_s.hvplot(title='Change me') * snow_selected.hvplot()  

#(marker='o',ax=ax1,color='orange') #label='Discharge', xlabel='', ylabel='Discharge (m$^3$/s)',ax=ax0) 
#ax0.legend(loc='center left', bbox_to_anchor=(0, 0.6))
#ax1.set_ylabel('Snow cover area (%)')
#ax1.legend(loc='center left', bbox_to_anchor=(0, 0.5),labels=['SCA'])
#plt.show()